In [1]:
!pip install mlflow
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, Subset
from random import sample
from torch.utils.data import DataLoader
from transformers import RobertaModel, RobertaTokenizer
from sklearn.utils import resample
from torchmetrics.classification import (
    MulticlassF1Score,
    MulticlassPrecision,
    MulticlassRecall,
)
from tqdm import tqdm
import mlflow
import time
import pandas as pd
import os

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.0/29.0 MB 65.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 105.3 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 722.9/722.9 kB 40.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.4/203.4 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 5.1 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 25.0
    Uninstalling packaging-25.0:
      Successfully uninstalled packaging-25.0
  Attempting uninstall: importlib_

2025-05-31 19:17:37.923257: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748719058.107504      75 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748719058.159509      75 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
LABEL_MAPPING = {
    "pants-fire": 0,
    "false": 1,
    "barely-true": 2,
    "half-true": 3,
    "mostly-true": 4,
    "true": 5,
}

ids2labels = [
    "pants-fire",
    "false",
    "barely-true",
    "half-true",
    "mostly-true",
    "true",
]


def save_checkpoint(model, optimizer, epoch, val_acc, path="checkpoint.pth"):
    checkpoint = {
        "model_state_dict": model.state_for_save(),
        "optimizer_state_dict": optimizer.state_dict(),
        "epoch": epoch,
        "val_acc": val_acc,
    }
    torch.save(checkpoint, path)
    print(
        f"Checkpoint saved at epoch {epoch} "
        f"with validation accuracy {val_acc:.4f}"
    )


def load_checkpoint(
    model, optimizer, path="checkpoint.pth", resume=False, reset_epoch=False
):
    if not resume:
        print("Resume is False. Starting from scratch.")
        return 0, 0  # Start fresh

    if os.path.exists(path):
        checkpoint = torch.load(path)
        model.load_state_from_save(checkpoint["model_state_dict"])
        optimizer.load_state_dict(checkpoint["optimizer_state_dict"])
        epoch = checkpoint["epoch"]
        val_acc = checkpoint["val_acc"]
        if reset_epoch:
            print(
                f"Checkpoint loaded: Starting from initial"
                f"epoch, validation accuracy {val_acc:.4f}"
            )
            return 0, val_acc  # Start fresh with existing model
        else:
            print(
                f"Checkpoint loaded: Resuming from epoch "
                f"{epoch+1}, validation accuracy {val_acc:.4f}"
            )
            return epoch + 1, val_acc  # Next epoch to train
    else:
        print("No checkpoint found. Starting from scratch.")
        return 0, 0  # Start fresh


def save_best_model(model, optimizer, epoch, val_acc, path="best_model.pth"):
    best_model = {
        "model_state_dict": model.state_for_save(),
        "optimizer_state_dict": optimizer.state_dict(),
        "epoch": epoch,
        "val_acc": val_acc,
    }
    torch.save(best_model, path)
    print(
        f"Best model saved at epoch {epoch} "
        f"with validation accuracy {val_acc:.4f}"
    )


def load_best_model(model, path="best_model.pth"):
    if os.path.exists(path):
        best_model = torch.load(path)
        model.load_state_from_save(best_model["model_state_dict"])
        print("Model loaded from best model checkpoint.")
    else:
        print("No best model checkpoint found.")

def save_model_remotely(local_path, remote_path, creds):
    pass


In [3]:
class PrepareDataset(Dataset):
    def __init__(
        self,
        filepath: str,
        num_metadata_col: str,
    ):
        self.df = pd.read_csv(filepath)
        self.num_metadata_col = num_metadata_col

    def __len__(self):
        return len(self.df.index)

    def __getitem__(self, index: int):
        item = self.df.iloc[index]
        column = self.num_metadata_col

        return {
            "num_metadata": torch.tensor([item[column]], dtype=torch.float32),
            "label": torch.tensor(LABEL_MAPPING[item["label"]]),
        }

In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class Classifier(nn.Module):
    def __init__(self, num_metadata_len, num_hidden, num_classes):
        super(Classifier, self).__init__()
        self.hl = nn.Linear(num_metadata_len, num_hidden)
        self.fc = nn.Linear(num_hidden, num_classes)

    def forward(self, num_metadata):
        hl_output = F.gelu(self.hl(num_metadata))
        logits = self.fc(hl_output)
        return logits

    def state_for_save(self):
        return {
            'hl_state_dict': self.hl.state_dict(),
            'fc_state_dict': self.fc.state_dict(),
        }
        
    def load_state_from_save(self, state):
        self.hl.load_state_dict(state['hl_state_dict'])
        self.fc.load_state_dict(state['fc_state_dict'])

In [5]:
def train(
    creds: dict,
    model: nn.Module,
    save_path: str,
    remote_models_path: str,
    best_model_path: str,
    train_loader: DataLoader,
    val_loader: DataLoader,
    test_loader: DataLoader,
    batch_size: int,
    num_classes: int,
    lr=1e-3,
    epochs=30,
    patience=5,
    resume: bool = False,
    reset_epoch: bool = False,
) -> None:
    dev_name = "cuda" if torch.cuda.is_available() else "cpu"
    print(f"Using device {dev_name}")
    device = torch.device(dev_name)

    # Define optimizer and loss function
    # Train only the classifier
    optimizer = torch.optim.Adam(model.fc.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()

    # Checkpoint Path
    checkpoint_path = f"checkpoint_{patience}.pth"

    checkpoint_send_interval = 2

    # Track best loss for model saving
    # Load Checkpoint (Decide if you want to continue)
    start_epoch, best_val_accuracy = load_checkpoint(
        model,
        optimizer,
        checkpoint_path,
        resume,
        reset_epoch
    )

    patience_counter = 0

    f1 = MulticlassF1Score(num_classes, average=None).to(device)
    precision = MulticlassPrecision(num_classes, average=None).to(device)
    recall = MulticlassRecall(num_classes, average=None).to(device)

    # Training loop
    for epoch in range(start_epoch, epochs):
        model.train()
        epoch_loss = 0

        train_accuracy = 0

        for batch in tqdm(
            train_loader, desc=f"Epoch {epoch+1}", leave=False
        ):
            num_metadata = batch["num_metadata"].to(device)
            labels = batch["label"].to(device)

            optimizer.zero_grad()
            outputs = model(num_metadata)
            loss = criterion(
                outputs, labels
            )  # można spróbować to logować jako osobny wykres do debugowania
            loss.backward()
            optimizer.step()

            epoch_loss += loss.item()

            # Calculate accuracy
            preds = torch.argmax(outputs, dim=-1)
            train_accuracy += (preds == labels).sum().item()

            f1.update(preds, labels)
            precision.update(preds, labels)
            recall.update(preds, labels)

        avg_loss = epoch_loss / len(train_loader)
        avg_train_accuracy = train_accuracy / len(train_loader.dataset)
        mlflow.log_metric("train_loss", avg_loss, step=epoch)
        mlflow.log_metric("train_acc", avg_train_accuracy, step=epoch)

        f1_res = f1.compute()
        precision_res = precision.compute()
        recall_res = recall.compute()

        for i in range(num_classes):
            mlflow.log_metric(
                f"train_f1_{ids2labels[i]}", f1_res[i], step=epoch
            )
            mlflow.log_metric(
                f"train_precision_{ids2labels[i]}",
                precision_res[i],
                step=epoch,
            )
            mlflow.log_metric(
                f"train_recall_{ids2labels[i]}", recall_res[i], step=epoch
            )

        macro_f1 = f1_res.mean()
        macro_precision = precision_res.mean()
        macro_recall = recall_res.mean()

        mlflow.log_metric("train_f1", macro_f1, step=epoch)
        mlflow.log_metric("train_precision", macro_precision, step=epoch)
        mlflow.log_metric("train_recall", macro_recall, step=epoch)

        tqdm.write(
            f"Epoch {epoch+1}: "
            f"Training Loss: {avg_loss}, "
            f"Training Accuracy: {avg_train_accuracy}, "
            f"Training F1: {macro_f1}, "
            f"Training Precision: {macro_precision}, "
            f"Training Recall: {macro_recall}"
        )

        # Validation step
        model.eval()  # Switch to evaluation mode
        val_loss = 0
        val_accuracy = 0

        f1.reset()
        precision.reset()
        recall.reset()

        with torch.no_grad():
            for batch in tqdm(
                val_loader,
                desc=f"Validation of epoch {epoch + 1}",
                leave=False,
            ):
                num_metadata = batch["num_metadata"].to(device)
                labels = batch["label"].to(device)

                outputs = model(num_metadata)
                loss = criterion(outputs, labels)
                val_loss += loss.item()

                # Calculate accuracy
                preds = torch.argmax(outputs, dim=-1)
                val_accuracy += (preds == labels).sum().item()
                f1.update(preds, labels)
                precision.update(preds, labels)
                recall.update(preds, labels)

        avg_val_loss = val_loss / len(val_loader)
        avg_val_accuracy = val_accuracy / len(val_loader.dataset)
        mlflow.log_metric("val_loss", avg_val_loss, step=epoch)
        mlflow.log_metric("val_acc", avg_val_accuracy, step=epoch)

        f1_res = f1.compute()
        precision_res = precision.compute()
        recall_res = recall.compute()

        for i in range(num_classes):
            mlflow.log_metric(
                f"val_f1_{ids2labels[i]}", f1_res[i], step=epoch
            )
            mlflow.log_metric(
                f"val_precision_{ids2labels[i]}",
                precision_res[i],
                step=epoch,
            )
            mlflow.log_metric(
                f"val_recall_{ids2labels[i]}", recall_res[i], step=epoch
            )

        macro_f1 = f1_res.mean()
        macro_precision = precision_res.mean()
        macro_recall = recall_res.mean()

        mlflow.log_metric("val_f1", macro_f1, step=epoch)
        mlflow.log_metric("val_precision", macro_precision, step=epoch)
        mlflow.log_metric("val_recall", macro_recall, step=epoch)

        print(
            f"Epoch {epoch+1}: "
            f"Validation Loss: {avg_val_loss}, "
            f"Validation Accuracy: {avg_val_accuracy}, "
            f"Validation F1: {macro_f1}, "
            f"Validation Precision: {macro_precision}, "
            f"Validation Recall: {macro_recall}"
        )

        save_checkpoint(
            model, optimizer, epoch, avg_val_accuracy, checkpoint_path
        )
        if (epoch + 1) % checkpoint_send_interval == 0:# and epoch != 0:
            save_model_remotely(checkpoint_path, remote_models_path, creds)

        # Check for early stopping
        if avg_val_accuracy > best_val_accuracy:
            best_val_accuracy = avg_val_accuracy
            patience_counter = 0
            # Save the best model
            save_best_model(
                model,
                optimizer,
                epoch,
                best_val_accuracy,
                best_model_path
            )
            save_model_remotely(best_model_path, remote_models_path, creds)
        else:
            patience_counter += 1
            if patience_counter >= patience:
                print(f"Early stopping at epoch {epoch+1}")
                break

    # Log final checkpoint
    save_model_remotely(checkpoint_path, remote_models_path, creds)

In [13]:
def test(
    model: nn.Module,
    best_model_path: str,
    dataloader: DataLoader
) -> None:
    # Define loss function
    criterion = nn.CrossEntropyLoss()

    load_best_model(model, best_model_path)
    
    model.eval()  # Set model to evaluation mode
    total_loss = 0.0
    total_correct = 0
    total_samples = 0

    f1 = MulticlassF1Score(num_classes, average=None).to(device)
    precision = MulticlassPrecision(num_classes, average=None).to(device)
    recall = MulticlassRecall(num_classes, average=None).to(device)

    with torch.no_grad():
        for batch in tqdm(dataloader, desc="Evaluating"):
            num_metadata = batch["num_metadata"].to(device)
            labels = batch["label"].to(device)

            outputs = model(num_metadata)
            loss = criterion(outputs, labels)
            total_loss += loss.item() * num_metadata.size(0)

            preds = torch.argmax(outputs, dim=1)
            total_correct += (preds == labels).sum().item()
            total_samples += num_metadata.size(0)

            f1.update(preds, labels)
            precision.update(preds, labels)
            recall.update(preds, labels)

    avg_loss = total_loss / total_samples
    accuracy = total_correct / total_samples

    f1_res = f1.compute()
    precision_res = precision.compute()
    recall_res = recall.compute()

    mlflow.log_metric("test_acc", accuracy)
    mlflow.log_metric("test_loss", avg_loss)

    for i in range(num_classes):
        mlflow.log_metric(f"test_f1_{ids2labels[i]}", f1_res[i])
        mlflow.log_metric(f"test_precision_{ids2labels[i]}", precision_res[i])
        mlflow.log_metric(f"test_recall_{ids2labels[i]}", recall_res[i])
    
    macro_f1 = f1_res.mean()
    macro_precision = precision_res.mean()
    macro_recall = recall_res.mean()

    mlflow.log_metric("test_f1", macro_f1)
    mlflow.log_metric("test_precision", macro_precision)
    mlflow.log_metric("test_recall", macro_recall)

    print(
        f"Test Loss: {avg_loss:.4f}, "
        f"Test Accuracy: {accuracy:.4f}, "
        f"Test F1: {f1_res} (marcro = {macro_f1:.4f}), "
        f"Test Precision: {precision_res} (marcro = {macro_precision:.4f}), "
        f"Test Recall: {recall_res} (marcro = {macro_recall:.4f}), "
    )

In [15]:
def do_experiment(training_data, validation_data, test_data):
    # MLflow experiment setup
    print(f"Starting ColumnTest_{num_column}")

    mlflow_uri = "http://cimmerian.win:5000"
    resume = False
    reset_epoch = False

    creds = {
        'hostname': "cimmerian.win",
        'port': 22,
        'username': "conan",
        'password': "conan"
    }
    
    mlflow.set_tracking_uri(uri=mlflow_uri)
    mlflow.set_experiment(f"ColumnTest_{num_column}")
        
    
        
    train_dataloader = DataLoader(
        training_data, batch_size=batch_size, shuffle=True
    )
    val_dataloader = DataLoader(
        validation_data, batch_size=batch_size, shuffle=True
    )
    test_dataloader = DataLoader(
        test_data, batch_size=batch_size, shuffle=True
    )
    
    # Instantiate model
    model = Classifier(
    num_metadata_len=1,  # jedna kolumna numeryczna
    num_hidden=hidden_size,
    num_classes=num_classes,
    )

    model.to(device)
    
    start = time.time()
    with mlflow.start_run():
        mlflow.log_param("learning_rate", lr)
        mlflow.log_param("batch_size", batch_size)
        mlflow.log_param("epochs", epochs)
        mlflow.log_param("resume", resume)
        mlflow.log_param("reset_epoch", reset_epoch)
        mlflow.log_param("patience", patience)
        
        # Train the model
        train(
            creds,
            model,
            save_path,
            remote_models_path,
            best_model_path,
            train_dataloader,
            val_dataloader,
            test_dataloader,
            batch_size,
            num_classes,
            lr,
            epochs,
            patience,
            resume,
            reset_epoch,
        )
        # Evaluate on test dataset
        test(model, best_model_path, test_dataloader)
    end = time.time()
    print(f"Total time took training: {end-start}s")

# Hyperparameters
num_classes = 6
lr = 1e-3
epochs = 64
hidden_size = 128
# Number of epochs to wait before stopping if no improvement
patience = 10

# Save path
save_path = "/kaggle/working"
# Remote models path
remote_models_path = "/home/conan/models/single_roberta/"
# Best model path
best_model_path = f"{save_path}/best_model_{patience}.pth"

batch_size = 64

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

num_metadata_cols = [
    # "barely_true_counts",
    # "false_counts",
    # "half_true_counts",
    # "mostly_true_counts",
    # "pants_on_fire_counts",
    "grammar_errors",
    # "ratio_of_capital_letters"
]    
for num_column in num_metadata_cols:    
    training_data = PrepareDataset(
        "/kaggle/input/normalized/train2.csv",
        num_metadata_col=num_column
    )
    validation_data = PrepareDataset(
        "/kaggle/input/normalized/val2.csv",
        num_metadata_col=num_column
    )
    test_data = PrepareDataset(
        "/kaggle/input/normalized/test2.csv",
        num_metadata_col=num_column
    )
    do_experiment(training_data, validation_data, test_data)



Starting ColumnTest_grammar_errors
Using device cuda
Resume is False. Starting from scratch.


Epoch 1: Training Loss: 1.764677557886017, Training Accuracy: 0.1983640081799591, Training F1: 0.12420692294836044, Training Precision: 0.13900673389434814, Training Recall: 0.16584360599517822


Epoch 1: Validation Loss: 1.7626305194128127, Validation Accuracy: 0.19003115264797507, Validation F1: 0.06935225427150726, Validation Precision: 0.05740497633814812, Validation Recall: 0.16774830222129822
Checkpoint saved at epoch 0 with validation accuracy 0.1900
Best model saved at epoch 0 with validation accuracy 0.1900


Epoch 2: Training Loss: 1.7587326987189535, Training Accuracy: 0.2003116174895316, Training F1: 0.11681190133094788, Training Precision: 0.15734918415546417, Training Recall: 0.16555066406726837


Epoch 2: Validation Loss: 1.7740600449698312, Validation Accuracy: 0.20482866043613707, Validation F1: 0.07379080355167389, Validation Precision: 0.06988987326622009, Validation Recall: 0.16784852743148804
Checkpoint saved at epoch 1 with validation accuracy 0.2048
Best model saved at epoch 1 with validation accuracy 0.2048


...

Epoch 11: Validation Loss: 1.7665294238499232, Validation Accuracy: 0.1939252336448598, Validation F1: 0.0698423832654953, Validation Precision: 0.06976013630628586, Validation Recall: 0.16668711602687836
Checkpoint saved at epoch 10 with validation accuracy 0.1939


Epoch 12: Training Loss: 1.758607702225632, Training Accuracy: 0.1999220956276171, Training F1: 0.12058179080486298, Training Precision: 0.12799586355686188, Training Recall: 0.16606056690216064


Epoch 12: Validation Loss: 1.7602514369147164, Validation Accuracy: 0.20482866043613707, Validation F1: 0.07379080355167389, Validation Precision: 0.06988987326622009, Validation Recall: 0.16784852743148804
Checkpoint saved at epoch 11 with validation accuracy 0.2048
Early stopping at epoch 12
Model loaded from best model checkpoint.


Evaluating: 100%|██████████| 21/21 [00:00<00:00, 158.35it/s]


Test Loss: 1.7517, Test Accuracy: 0.1902, Test F1: tensor([0.0000, 0.3208, 0.0852, 0.0000, 0.0000, 0.0000], device='cuda:0') (marcro = 0.0677), Test Precision: tensor([0.0000, 0.1941, 0.1429, 0.0000, 0.0000, 0.0000], device='cuda:0') (marcro = 0.0562), Test Recall: tensor([0.0000, 0.9240, 0.0607, 0.0000, 0.0000, 0.0000], device='cuda:0') (marcro = 0.1641), 
🏃 View run treasured-snake-600 at: http://cimmerian.win:5000/#/experiments/24/runs/3583552bae9745788f8c305113db3914
🧪 View experiment at: http://cimmerian.win:5000/#/experiments/24
Total time took training: 224.86124205589294s
